In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.getcwd())
import numpy as np
from matplotlib import pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub


tf.reset_default_graph()
dim_z = 120


DATA_DIR = '__place_path_to_images_root__'
EMBEDDINGS_FILE = '__output_file__'


# Load BigBiGAN module.
module = hub.Module('https://tfhub.dev/deepmind/bigbigan-resnet50/1')  # small encoder
# module = hub.Module('https://tfhub.dev/deepmind/bigbigan-revnet50x4/1')  # large encoder

z = tf.random.truncated_normal([24, 120])  # latent samples
gen_samples = module(z, signature='generate')

images = tf.placeholder(tf.float32, shape=[None, 256, 256, 3])
features = module(images, signature='encode', as_dict=True)
z_sample = features['z_sample']  # shape [?, 120]

recons = module(z_sample, signature='generate')  # shape [?, 128, 128, 3]


sess = tf.InteractiveSession()
initializer = tf.global_variables_initializer()
sess.run(initializer)

In [ ]:
from torchvision import transforms
from data import UnannotatedDataset

ds = UnannotatedDataset(
    DATA_DIR,
    transform=transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(256),
        transforms.ToTensor(),
        lambda x: (2.0 * x - 1.0).permute(1, 2, 0).numpy()
    ]))


gen_np = np.stack([ds[i] for i in range(0, 10)])
recons_np = sess.run(recons, feed_dict={images: gen_np})

plt.figure(figsize=(10, 2), dpi=200)
batch = gen_np.shape[0]

for i in range(batch):
    plt.subplot(2, len(gen_np), i + 1)
    plt.axis('off')
    plt.imshow(0.5 * gen_np[i] + 0.5)

    plt.subplot(2, len(gen_np), len(gen_np) + i + 1)
    plt.axis('off')
    plt.imshow(0.5 * recons_np[i] + 0.5)

In [ ]:
from tqdm import tqdm_notebook

batch_size = 100
emdeddings = []

all_indices = np.arange(0, len(ds))
count = len(all_indices)
for batch_start in tqdm_notebook(np.arange(0, count, batch_size), total=count // batch_size):
    low, up = batch_start, min(batch_start + batch_size, count)
    gen_np = np.stack([ds[i] for i in all_indices[low: up]])
    emdeddings.append(sess.run(features, feed_dict={images: gen_np}))

In [ ]:
zs = [e['z_sample'] for e in emdeddings]
zs = np.vstack(zs)
np.save(EMBEDDINGS_FILE, zs)